# Generate margin caches

Let's generate margin cache catalogs for our nested data products, `diaObject_lc` and `object_lc`.

In [1]:
%pip uninstall hats-import -y
%pip install git+https://github.com/astronomy-commons/hats-import.git@issue/465/margin-cache-nested-catalogs

Found existing installation: hats-import 0.4.5.dev4+g222830b
Uninstalling hats-import-0.4.5.dev4+g222830b:
  Successfully uninstalled hats-import-0.4.5.dev4+g222830b
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/astronomy-commons/hats-import.git (to revision issue/465/margin-cache-nested-catalogs) to /lscratch/stavar/tmp/pip-req-build-whl0nla_
  Running command git clone --filter=blob:none --quiet https://github.com/astronomy-commons/hats-import.git /lscratch/stavar/tmp/pip-req-build-whl0nla_
  Running command git checkout -b issue/465/margin-cache-nested-catalogs --track origin/issue/465/margin-cache-nested-catalogs
  Switched to a new branch 'issue/465/margin-cache-nested-catalogs'
  branch 'issue/465/margin-cache-nested-catalogs' set up to track 'origin/issue/465/margin-cache-nested-catalogs'.
  Resolved https://github.com/astronomy-commons/hats-import.git t

In [2]:
import hats_import
print(hats_import.__version__)

0.4.5.dev4+g222830b


In [3]:
import os
import tempfile
import hats_import.pipeline as runner

from dask.distributed import Client
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from pathlib import Path

In [4]:
margin_radius_arcsec = 5

In [5]:
DRP_VERSION = os.environ["DRP_VERSION"]
print(f"DRP_VERSION: {DRP_VERSION}")
base_output_dir = Path(f"/sdf/data/rubin/shared/lsdb_commissioning")
hats_dir = base_output_dir / "hats" / DRP_VERSION

DRP_VERSION: w_2025_08


In [6]:
tmp_path = tempfile.TemporaryDirectory()
tmp_dir = tmp_path.name
client = Client(n_workers=4, threads_per_worker=1, local_directory=tmp_dir)

In [7]:
args = MarginCacheArguments(
    input_catalog_path=hats_dir / "diaObject_lc",
    output_path=hats_dir,
    margin_threshold=margin_radius_arcsec,
    output_artifact_name=f"diaObject_lc_{margin_radius_arcsec}arcs",
)
runner.pipeline_with_client(args, client)

Planning  :   0%|          | 0/3 [00:00<?, ?it/s]

Mapping   :   0%|          | 0/251 [00:00<?, ?it/s]

Binning   :   0%|          | 0/1 [00:00<?, ?it/s]

Reducing  :   0%|          | 0/342 [00:00<?, ?it/s]

Finishing :   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
args = MarginCacheArguments(
    input_catalog_path=hats_dir / "object_lc",
    output_path=hats_dir,
    margin_threshold=margin_radius_arcsec,
    output_artifact_name=f"object_lc_{margin_radius_arcsec}arcs",
)
runner.pipeline_with_client(args, client)

Planning  :   0%|          | 0/3 [00:00<?, ?it/s]

Mapping   :   0%|          | 0/212 [00:00<?, ?it/s]

Binning   :   0%|          | 0/1 [00:00<?, ?it/s]

Reducing  :   0%|          | 0/324 [00:00<?, ?it/s]

Finishing :   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
client.close()
tmp_path.cleanup()